# SurvivalStan

Tutorial: http://www.hammerlab.org/2017/06/26/introducing-survivalstan/

TODO

- [ ] Simulate Exponential model with covariates: 1 + age + sex the correct form?
- [ ] Try to reduce divergeces
- [ ] Why do coefficients not match?

## Set up environment

In [ ]:
# Install Stan utilities

!rm stan_utility.py
!wget https://raw.githubusercontent.com/betanalpha/jupyter_case_studies/master/pystan_workflow/stan_utility.py
    

# Upgrade pip
!pip3 install --user --upgrade pip


# Install PyStan
!pip3 install --user pystan


# Install SurvivalStan
!git clone https://github.com/hammerlab/survivalstan.git
!pip3 uninstall --yes survivalstan
!pip3 install --user survivalstan/.

In [ ]:
import sys
import os

import numpy as np
import pandas as pd

%matplotlib inline
from matplotlib import pyplot as plt

import pystan
import survivalstan
import stan_utility

## Simple exponential model with covariates

This serves as a simple sanity check.

$S(t) = \exp( -at ) \exp( \beta \cdot X )$

The following prior strategy will be used:

* Priors for established risk factors
* Priors for factor with more uncertain relevance

In [ ]:
# Simulate Data
df2 = survivalstan.sim.sim_data_exp_correlated(
    N=500, censor_time=20, rate_form='1 + sex', rate_coefs=[-3, 0.5])


survivalstan.utils.plot_observed_survival(
    df2.query('sex == "female"'),
    time_col='t',
    event_col='event',
    label='female')

survivalstan.utils.plot_observed_survival(
    df2.query('sex == "male"'),
    time_col='t',
    event_col='event',
    label='male')

plt.legend()

In [ ]:
# Infer parameters

ctrl = dict(adapt_delta = 0.9,
            max_treedepth = 20)

fit2_org = survivalstan.fit_stan_survival_model(
    df=df2,
    time_col='t',
    event_col='event',
    model_code=survivalstan.models.exp_survival_model,
    formula='~ age + sex',
    model_cohort = 'exp model',
    iter = 10000,
    chains = 4,
    warmup = 9500,
    control = ctrl)

In [ ]:
# Results
survivalstan.utils.filter_stan_summary([fit2_org], pars=['lp__','beta'])

sfit2_org = fit2_org['fit']
sfit2_org

In [ ]:
# Plot divergent transitions
light="#DCBCBC"
light_highlight="#C79999"
mid="#B97C7C"
mid_highlight="#A25050"
dark="#8F2727"
dark_highlight="#7C0000"
green="#00FF00"

nondiv_params, div_params = stan_utility.partition_div(sfit2_org)

plt.scatter([x[0] for x in nondiv_params['beta']], 
             [x[1] for x in nondiv_params['beta']], \
             color = mid_highlight, alpha=0.05)
plt.scatter([x[0] for x in div_params['beta']],
             [x[1] for x in div_params['beta']], \
             color = green, alpha=0.5)

plt.gca().set_xlabel("beta_1")
plt.gca().set_ylabel("beta_2")

plt.show()

In [ ]:
# HMC diagnostics

sfit2_org = fit2_org['fit']

pystan.check_hmc_diagnostics(sfit2_org)

print(sfit2_org)

## More flexible baseline hazards: Piecewise hazard

One of the more critical parameterizations to get right is that of the baseline hazard. The baseline hazard behaves like an intercept in a typical regression model. It describes the instantaneous hazard over time for the population in the absence of any covariate effects. Failure to get this right can lead to all sorts of pathologies whereby the excess variation in hazard not accounted for by your modeled baseline hazard will be absorbed into covariate effects, yielding invalid inferences and potentially misleading conclusions.

*Aside: This is not a concern when using a Cox PH model for example, because the coefficient values are estimated using Maximum Likelihood Estimation (MLE) on a partial likelihood which does not include the baseline hazard. In a Bayesian analysis, however, we have the challenge of estimating the hazard as well as the coefficient effects.*

Most of the time, we do not have a prior belief on the distribution of the baseline hazard. We usually do not care that much about what the features of the baseline hazard look like (although perhaps we should!). Instead, we are concerned with making sure our inferences about coefficient values are valid.

We thus want a baseline hazard that is sufficiently flexible to absorb any variation in the hazard over time which should not be attributed to covariate values. We also however want to minimize the risk of overfitting, so that our posterior predicted probabilities of survival are well calibrated. Many of the semi- or non-parametric approaches to modeling baseline hazards are very flexible with a penalty to impose the upper bound of complexity.

Piecewise-Exponential Model: https://data.princeton.edu/wws509/notes/c7s4